In [ ]:
!sudo service nvargus-daemon restart

In [ ]:
from jduck.utils.logging_config import logger as logging

In [2]:
logging.info('Hello')

  210308-12:59|<ipython-input-2-1e24994359dc>:1|INFO|  Hello


In [4]:
from jduck.robot import JDuck

ModuleNotFoundError: No module named 'DCMotor'

In [1]:
from jduck.camera.opencv_gst_camera import *

In [ ]:
from jduck import Camera

In [ ]:
camera = OpenCvGstCamera.instance()

In [6]:
from jduck.image import bgr8_to_jpeg

In [7]:
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

NameError: name 'camera' is not defined

In [6]:
# Standard Packages
import os

# Utilities
import traitlets


class CameraBase(traitlets.HasTraits):

    value = traitlets.Any()
    def __init__(self, *args, **kwargs):
        super().__init__(self, *args, **kwargs)

    @staticmethod
    def instance(*args, **kwargs):
        raise NotImplementedError

    def widget(self):
        if hasattr(self, "_widget"):
            return self._widget  # cache widget, so we don't duplicate links
        from ipywidgets import Image
        from jetbot.image import bgr8_to_jpeg

        image = Image()
        traitlets.dlink((self, "value"), (image, "value"), transform=bgr8_to_jpeg)
        self._widget = image
        return image
# Standard Packages
import atexit
import threading

# Data Analytics
import numpy as np

# Visualization Packages
import cv2

class OpenCvGstCamera(CameraBase):

    value = traitlets.Any()

    # config
    flip_method = traitlets.Integer(default_value=0).tag(config=True)
    width = traitlets.Integer(default_value=224).tag(config=True)
    height = traitlets.Integer(default_value=224).tag(config=True)
    fps = traitlets.Integer(default_value=30).tag(config=True)
    capture_width = traitlets.Integer(default_value=816).tag(config=True)
    capture_height = traitlets.Integer(default_value=616).tag(config=True)

    def __init__(self, *args, **kwargs):
        self.value = np.empty((self.height, self.width, 3), dtype=np.uint8)
        super().__init__(self, *args, **kwargs)

        try:
            self.cap = cv2.VideoCapture(self._gst_str(), cv2.CAP_GSTREAMER)

            re, image = self.cap.read()

            if not re:
                raise RuntimeError("Could not read image from camera.")

            self.value = image
            self.start()
        except:
            self.stop()
            raise RuntimeError("Could not initialize camera.  Please see error trace.")

        atexit.register(self.stop)

    def _capture_frames(self):
        while True:
            re, image = self.cap.read()
            if re:
                self.value = image
            else:
                break

    def _gst_str(self):
        return (
            "nvarguscamerasrc ! "
            "video/x-raw(memory:NVMM), "
            "width=(int)%d, height=(int)%d, "
            "format=(string)NV12, framerate=(fraction)%d/1 ! "
            "nvvidconv flip-method=%d ! "
            "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
            "videoconvert ! "
            "video/x-raw, format=(string)BGR ! appsink"
            % (
                self.capture_width,
                self.capture_height,
                self.fps,
                self.flip_method,
                self.width,
                self.height,
            )
        )

    def start(self):
        if not self.cap.isOpened():
            self.cap.open(self._gst_str(), cv2.CAP_GSTREAMER)
        if not hasattr(self, "thread") or not self.thread.isAlive():
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if hasattr(self, "cap"):
            self.cap.release()
        if hasattr(self, "thread"):
            self.thread.join()

    def restart(self):
        self.stop()
        self.start()

    @staticmethod
    def instance(*args, **kwargs):
        return OpenCvGstCamera(*args, **kwargs)

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [8]:
camera = OpenCvGstCamera.instance()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Passing unrecognized arguments to super(OpenCvGstCamera).__init__(<__main__.OpenCvGstCamera object at 0x7f9c51f358>, <__main__.OpenCvGstCamera object at 0x7f9c51f358>).
object.__init__() takes no parameters
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  if sys.path[0] == '':


RuntimeError: Could not initialize camera.  Please see error trace.

In [7]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

camera = OpenCvGstCamera.instance()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Passing unrecognized arguments to super(OpenCvGstCamera).__init__(<__main__.OpenCvGstCamera object at 0x7f98c6d358>, <__main__.OpenCvGstCamera object at 0x7f98c6d358>).
object.__init__() takes no parameters
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  if sys.path[0] == '':


RuntimeError: Could not initialize camera.  Please see error trace.

In [1]:
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

NameError: name 'widgets' is not defined